In [1]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy.spatial.distance import cdist
from skimage import measure
from matplotlib import pyplot as plt
import os
import tifffile
from tqdm import tqdm
from typing import List, Tuple

In [2]:
selected_magnification = "20x"
selected_image_type = "raw"
max_dist = 9
aspect = np.array((1/3, 1/3))    # XY: 20x / aspect => 60x

In [3]:
dot_image_folder_path = f"/mnt/data/Imaging/202105-Deconwolf/data_210726/{selected_magnification}_{selected_image_type}"
ref_image_folder_path = "/mnt/data/Imaging/202105-Deconwolf/data_210726/60x_dw"
dot_mask_folder_path = "../../data/20x_mask/dilated_labels_watershed_from60x"
ref_mask_folder_path = "../../data/60x_mask/dilated_labels_watershed"

### Read shift data

In [4]:
shifts = pd.read_csv(f"shift_correction/{selected_magnification}_{selected_image_type}.shifts.csv")
shifts.index = shifts["sid"].values
shifts.drop("sid", 1, inplace=True)

# Matching 20x_raw and reference dots

In [5]:
dots_data = pd.read_csv("/mnt/data/Imaging/202105-Deconwolf/data_210726/dots_data.clean.tsv.gz", sep="\t")
dots_data = dots_data[selected_magnification == dots_data["magnification"]]
dots_data = dots_data[selected_image_type == dots_data["image_type"]]

In [6]:
thresholds_table = pd.read_csv("../../data/magnifications_matching/intensity_thresholds.by_field.tsv", sep="\t")

In [7]:
matched_dots = pd.read_csv(
    os.path.join("../../data/magnifications_matching",
                 f"{selected_magnification}_{selected_image_type}.matched_dots.field_thr.tsv"
                ), sep="\t")

In [8]:
reference = pd.read_csv("../../data/60x_reference/ref__dw.field_thr.tsv", sep="\t")

In [9]:
selected_20x_dots: List[pd.DataFrame] = []
selected_20x_dots_shifted: List[pd.DataFrame] = []
for current_field_id in tqdm(np.unique(dots_data["sid"])):
    thresholds = thresholds_table.loc[current_field_id == thresholds_table["sid"], :]
    intensity_thr = thresholds.loc[selected_image_type == thresholds["image_type"], "thr"].values[0]
    
    dot_max_z_proj = tifffile.imread(os.path.join(dot_image_folder_path, f"a647_{current_field_id:03d}.tif")).max(0)
    ref_max_z_proj = tifffile.imread(os.path.join(ref_image_folder_path, f"a647_{current_field_id:03d}.tif")).max(0)
    
    dot_labels = tifffile.imread(os.path.join(dot_mask_folder_path, f"a647_{current_field_id:03d}.dilated_labels.from_60x.tiff")
                                ).reshape(dot_max_z_proj.shape)
    ref_labels = tifffile.imread(os.path.join(ref_mask_folder_path, f"a647_{current_field_id:03d}.dilated_labels.tiff")
                                ).reshape(ref_max_z_proj.shape)
    
    dots = dots_data.loc[current_field_id == dots_data["sid"], :].copy(
        ).sort_values("Value2", ascending=False).reset_index(drop=True)
    
    dot_coords = dots.loc[intensity_thr <= dots["Value2"], ("x", "y")].copy().reset_index(drop=True)
    dot_coords2 = dot_coords.copy() / aspect
    dot_coords2["x"] += (shifts.loc[current_field_id, "x"] * 9)
    dot_coords2["y"] += (shifts.loc[current_field_id, "y"] * 9)
    ref_coords = reference.loc[reference["sid"] == current_field_id, ("x", "y")].copy().reset_index(drop=True)
    
    matched_20x_dots = matched_dots.loc[matched_dots["series"] == current_field_id, "id_20x"].values
    matched_60x_dots = matched_dots.loc[matched_dots["series"] == current_field_id, "id_60x"].values
    max_match_dist = matched_dots.loc[matched_dots["series"] == current_field_id, "eudist"].max()

    selected_20x_dots.append(dot_coords.loc[matched_20x_dots, :])
    selected_20x_dots_shifted.append(dot_coords2.loc[matched_20x_dots, :])

100%|██████████| 7/7 [00:04<00:00,  1.67it/s]


In [10]:
pd.concat(selected_20x_dots).reset_index(
    drop=True).to_csv(os.path.join("../../data/magnifications_matching",
                                   f"{selected_magnification}_{selected_image_type}.field_thr.tsv"),
                      sep="\t", index=False)

In [11]:
pd.concat(selected_20x_dots_shifted).reset_index(
    drop=True).to_csv(os.path.join("../../data/magnifications_matching",
                                   f"{selected_magnification}_{selected_image_type}.field_thr.shifted.tsv"),
                      sep="\t", index=False)